**Experiment-5: Decision Trees from Scratch**

Name : Amishi Gupta

Roll No. : 23/CS/048

In [45]:
!pip install ucimlrepo

In [46]:
#imports
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTree
import copy

1. Data Preparation

In [47]:
#Fetch dataset
try:
    adult = fetch_ucirepo(id=2)
    X_df = adult.data.features
    y_df = adult.data.targets
except Exception as e:
    print(f"Could not fetch data from ucimlrepo. Error:{e}")
    exit()


In [48]:
# combine features and target
df = pd.concat([X_df,y_df],axis=1)
df.replace('?', np.nan,inplace=True)

In [49]:
# handling missing values
df.dropna(inplace=True)
print(f"dataset shape after dropping missing values: {df.shape}")

dataset shape after dropping missing values: (45222, 15)


In [50]:
# separate features and target
X = df.drop('income',axis=1)
y = df['income']

In [51]:
# encode categorical features
categorical_cols=X.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le =LabelEncoder()
    X[col] =le.fit_transform(X[col])

In [52]:
#<=50K becomes 0,>50K becomes 1
y = y.apply(lambda x: 1 if x in ['>50K', '>50K.'] else 0)

In [53]:
X_np = X.to_numpy()
y_np = y.to_numpy()

In [54]:
# split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X_np, y_np, test_size=0.4, random_state=42, stratify=y_np)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [55]:
#checking
print(f"Training set size:{len(X_train)}")
print(f"Validation set size:{len(X_val)}")
print(f"Test set size:{len(X_test)}")

Training set size:27133
Validation set size:9044
Test set size:9045


2. Build a Decision Tree From Scratch and pre prunnung

In [33]:
#node
class Node:
    def __init__(self, feature_index=None,threshold=None,left=None,right=None,*,value=None,impurity=None):
        self.feature_index =feature_index
        self.threshold =threshold
        self.left =left
        self.right =right
        self.value= value
        self.impurity= impurity


In [64]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100,criterion='gini'):
        self.min_samples_split = min_samples_split
        self.max_depth =max_depth
        self.criterion =criterion
        self.root = None
        self.feature_names = None

    def _calculate_impurity(self, y):
        if self.criterion == 'gini':
            return self._gini_impurity(y)
        elif self.criterion =='entropy':
            return self._entropy(y)
        else:
            raise ValueError("Criterion must be 'gini' or 'entropy'")

    def _gini_impurity(self, y):
        if len(y) == 0:
            return 0
        _, counts =np.unique(y, return_counts=True)
        probabilities =counts/ len(y)
        return 1 - np.sum(probabilities**2)

    def _entropy(self, y):
        if len(y) == 0:
            return 0
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    def _information_gain(self, parent, left_child, right_child):
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        gain = self._calculate_impurity(parent) - \
               (weight_left * self._calculate_impurity(left_child) +
                weight_right * self._calculate_impurity(right_child))
        return gain

    def _find_best_split(self, X, y):
        best_gain = -1
        best_split = {}
        n_samples, n_features =X.shape

        for feature_index in range(n_features):
            feature_values = X[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            for threshold in possible_thresholds:
                left_indices = np.where(feature_values <= threshold)[0]
                right_indices = np.where(feature_values > threshold)[0]

                if len(left_indices) > 0 and len(right_indices) > 0:
                    y_left, y_right = y[left_indices], y[right_indices]
                    gain = self._information_gain(y, y_left, y_right)

                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
        return best_split

    def _build_tree(self, X, y, depth=0):
        n_samples, _ = X.shape
        n_labels = len(np.unique(y))

        # Pre-pruning stopping criteria
        if (self.max_depth is not None and depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, impurity=self._calculate_impurity(y))

        best_split = self._find_best_split(X, y)

        if not best_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, impurity=self._calculate_impurity(y))

        left_indices, right_indices = best_split['left_indices'], best_split['right_indices']

        left_subtree = self._build_tree(X[left_indices, :], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices, :], y[right_indices], depth + 1)

        return Node(best_split['feature_index'], best_split['threshold'],
                    left_subtree, right_subtree, impurity=self._calculate_impurity(y))

    def _most_common_label(self, y):
        if len(y) == 0:
            return None
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def fit(self, X, y, feature_names=None):
        self.feature_names = feature_names
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        feature_value = x[node.feature_index]
        if feature_value <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    def print_tree(self, node=None, indent="  ", level=0):
        if node is None:
            node = self.root

        if node.value is not None:
            print(indent * level, "Predict:", node.value, f"(Impurity: {node.impurity:.3f})")
        else:
            feature_name = self.feature_names[node.feature_index] if self.feature_names else f"Feature {node.feature_index}"
            print(indent * level, f"If {feature_name} <= {node.threshold}: (Impurity: {node.impurity:.3f})")
            self.print_tree(node.left, indent, level + 1)
            print(indent * level, f"Else (If {feature_name} > {node.threshold}):")
            self.print_tree(node.right, indent, level + 1)

print("Decision Tree Implementation Defined\n")


Decision Tree Implementation Defined



4. Post-Pruning (Reduced Error Pruning)

In [59]:
def post_prune(tree, X_val, y_val):

    pruned_tree = copy.deepcopy(tree)

    y_pred_initial = pruned_tree.predict(X_val)
    best_accuracy = accuracy_score(y_val, y_pred_initial)

    while True:
        current_best_prune_info = None

        queue = [(pruned_tree.root, None, None)]
        internal_nodes_with_parents = []
        while queue:
            current_node, parent, is_left = queue.pop(0)
            if current_node.value is None:
                internal_nodes_with_parents.append((current_node, parent, is_left))
                if current_node.left:
                    queue.append((current_node.left, current_node, True))
                if current_node.right:
                    queue.append((current_node.right, current_node, False))


        for node, parent, is_left in reversed(internal_nodes_with_parents):
            original_left, original_right = node.left, node.right
            original_feature, original_threshold = node.feature_index, node.threshold


            leaf_value_pred = Counter(pruned_tree.predict(X_train)).most_common(1)[0][0]
            node.value = leaf_value_pred
            node.left, node.right = None, None
            node.feature_index, node.threshold = None, None

            y_pred_pruned = pruned_tree.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred_pruned)


            if accuracy >= best_accuracy:
                best_accuracy = accuracy

                current_best_prune_info = (node, parent, is_left, copy.deepcopy(node))


            node.value = None
            node.left, node.right = original_left, original_right
            node.feature_index, node.threshold = original_feature, original_threshold


        if current_best_prune_info:
            node_to_prune, parent, is_left, new_leaf_node = current_best_prune_info

            if parent is None:
                 pruned_tree.root = new_leaf_node
            elif is_left:
                parent.left = new_leaf_node
            else:
                parent.right = new_leaf_node
        else:
            break

    return pruned_tree

print("Post-Pruning Implementation Defined.\n")


Post-Pruning Implementation Defined.



5. Evaluation

In [60]:
def evaluate_model(model, X_set, y_set, model_name="Model"):
    y_pred = model.predict(X_set)
    accuracy = accuracy_score(y_set, y_pred)
    precision, recall, f1, _ =precision_recall_fscore_support(y_set, y_pred,average='binary')
    cm = confusion_matrix(y_set, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("-" * 30 + "\n")
    return accuracy

6. Experiments

In [61]:
print("Experiment 1: Gini vs. Entropy (on Validation Set)")
gini_tree = DecisionTree(max_depth=10, criterion='gini')
gini_tree.fit(X_train, y_train)
evaluate_model(gini_tree, X_val, y_val,"Gini Impurity Tree")

entropy_tree = DecisionTree(max_depth=10,criterion='entropy')
entropy_tree.fit(X_train, y_train)
evaluate_model(entropy_tree, X_val, y_val,"Entropy Tree")


Experiment 1: Gini vs. Entropy (on Validation Set)
Accuracy: 0.8445
Precision: 0.7086
Recall: 0.6328
F1-score: 0.6686
Confusion Matrix:
[[6220  583]
 [ 823 1418]]
------------------------------

Accuracy: 0.8464
Precision: 0.7696
Recall: 0.5426
F1-score: 0.6365
Confusion Matrix:
[[6439  364]
 [1025 1216]]
------------------------------



0.8464175143741707

In [67]:
print("\nExperiment 2: Effect of Max Depth (Pre-Pruning on Validation Set)")
depths = [2, 4, 6, 8, 10, None] # None means unlimited depth
best_depth = -1
best_val_accuracy = -1
pre_pruned_tree = None

for depth in depths:
    current_depth_str = "Unlimited" if depth is None else str(depth)
    tree = DecisionTree(max_depth=depth, min_samples_split=5, criterion='gini')
    tree.fit(X_train, y_train)
    val_accuracy = evaluate_model(tree, X_val, y_val, f"Tree with Max Depth = {current_depth_str}")
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_depth = depth
        pre_pruned_tree = tree

print(f"Best depth found: {best_depth} with validation accuracy: {best_val_accuracy:.4f}\n")



Experiment 2: Effect of Max Depth (Pre-Pruning on Validation Set)
Accuracy: 0.8205
Precision: 0.7480
Recall: 0.4159
F1-score: 0.5346
Confusion Matrix:
[[6489  314]
 [1309  932]]
------------------------------

Accuracy: 0.8383
Precision: 0.7551
Recall: 0.5145
F1-score: 0.6120
Confusion Matrix:
[[6429  374]
 [1088 1153]]
------------------------------

Accuracy: 0.8479
Precision: 0.7729
Recall: 0.5466
F1-score: 0.6404
Confusion Matrix:
[[6443  360]
 [1016 1225]]
------------------------------

Accuracy: 0.8485
Precision: 0.7776
Recall: 0.5444
F1-score: 0.6404
Confusion Matrix:
[[6454  349]
 [1021 1220]]
------------------------------

Accuracy: 0.8444
Precision: 0.7083
Recall: 0.6328
F1-score: 0.6684
Confusion Matrix:
[[6219  584]
 [ 823 1418]]
------------------------------

Accuracy: 0.8046
Precision: 0.6023
Recall: 0.6225
F1-score: 0.6122
Confusion Matrix:
[[5882  921]
 [ 846 1395]]
------------------------------

Best depth found: 8 with validation accuracy: 0.8485



In [66]:
print("\nExperiment 3: Comparing Pruning Methods (on Validation Set)")
full_tree = DecisionTree(criterion='gini', max_depth=100)
full_tree.fit(X_train, y_train, feature_names=X.columns)
evaluate_model(full_tree, X_val,y_val, "Full (Unpruned) Tree")

evaluate_model(pre_pruned_tree, X_val, y_val,f"Pre-Pruned Tree (Best Depth={best_depth})")
post_pruned_tree_model = post_prune(full_tree, X_val, y_val)
evaluate_model(post_pruned_tree_model, X_val, y_val, "Post-Pruned Tree")


Experiment 3: Comparing Pruning Methods (on Validation Set)
Accuracy: 0.8035
Precision: 0.5999
Recall: 0.6216
F1-score: 0.6106
Confusion Matrix:
[[5874  929]
 [ 848 1393]]
------------------------------

Accuracy: 0.8485
Precision: 0.7776
Recall: 0.5444
F1-score: 0.6404
Confusion Matrix:
[[6454  349]
 [1021 1220]]
------------------------------



KeyboardInterrupt: 

In [68]:
print("\nFinal Evaluation on Test Set")
print("Choosing the Pre-Pruned Tree as the best model based on validation performance.")
evaluate_model(pre_pruned_tree, X_test, y_test, "Final Model (Pre-Pruned Scratch)")


Final Evaluation on Test Set
Choosing the Pre-Pruned Tree as the best model based on validation performance.
Accuracy: 0.8489
Precision: 0.7888
Recall: 0.5330
F1-score: 0.6361
Confusion Matrix:
[[6483  320]
 [1047 1195]]
------------------------------



0.8488667772249862

In [69]:
print("\nComparison with Scikit-learn")
sklearn_tree = SklearnDecisionTree(criterion='gini', max_depth=best_depth, min_samples_split=5,random_state=42)
sklearn_tree.fit(X_train, y_train)
evaluate_model(sklearn_tree, X_test, y_test, "Scikit-learn Decision Tree")


Comparison with Scikit-learn
Accuracy: 0.8494
Precision: 0.7902
Recall: 0.5343
F1-score: 0.6376
Confusion Matrix:
[[6485  318]
 [1044 1198]]
------------------------------



0.8494195688225539

In [70]:
print("\nTop Features from Scratch Implementation")
print("The most important features are those used for splits at the top of the tree.")
full_tree.print_tree(level=0)


Top Features from Scratch Implementation
The most important features are those used for splits at the top of the tree.


ValueError: The truth value of a Index is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().